In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.types import IntegerType
from pyspark.sql.window import Window
import pyspark.sql.functions as F

In [45]:
spark = SparkSession.builder.getOrCreate()
spark.sparkContext.setLogLevel("WARN")

In [87]:
df = spark.read.csv (
    "Border_Crossing_Entry_Data.csv",
    sep = ",",
    header = True,
    inferSchema = True,
)

In [88]:
df.show()

+-------------+------------+---------+----------------+--------+--------------------+------+--------+---------+--------------------+
|    Port Name|       State|Port Code|          Border|    Date|             Measure| Value|Latitude|Longitude|               Point|
+-------------+------------+---------+----------------+--------+--------------------+------+--------+---------+--------------------+
|      Jackman|       Maine|      104|US-Canada Border|Jan 2024|              Trucks|  6556|  45.806|  -70.397|POINT (-70.396722...|
|     Porthill|       Idaho|     3308|US-Canada Border|Apr 2024|              Trucks|    98|    49.0| -116.499|POINT (-116.49925...|
|     San Luis|     Arizona|     2608|US-Mexico Border|Apr 2024|               Buses|    10|  32.485| -114.782|POINT (-114.78222...|
| Willow Creek|     Montana|     3325|US-Canada Border|Jan 2024|         Pedestrians|     2|    49.0| -109.731|POINT (-109.73133...|
|      Warroad|   Minnesota|     3423|US-Canada Border|Jan 2024|Perso

In [89]:
# Rename columns
df = df.withColumnRenamed("Port Name", "Port_Name")

df = df.withColumnRenamed("Port Code", "Port_Code")

# Rename values to crossing count
df = df.withColumnRenamed("Value", "Crossing_Count")

In [90]:
# Reformat the date
df = df.withColumn("Date", F.date_format(F.to_date(F.col("Date"), "MMM yyyy"), "MM-yyyy"))

In [91]:
# Remove Unnecessary Columns
df_2 = df.drop("Point")

In [92]:
# Create Table
df_2.createOrReplaceTempView("Border_Table")

In [93]:
df_2.printSchema()

root
 |-- Port_Name: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Port_Code: integer (nullable = true)
 |-- Border: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- Measure: string (nullable = true)
 |-- Crossing_Count: integer (nullable = true)
 |-- Latitude: double (nullable = true)
 |-- Longitude: double (nullable = true)



In [94]:
df_2.show()

+-------------+------------+---------+----------------+-------+--------------------+--------------+--------+---------+
|    Port_Name|       State|Port_Code|          Border|   Date|             Measure|Crossing_Count|Latitude|Longitude|
+-------------+------------+---------+----------------+-------+--------------------+--------------+--------+---------+
|      Jackman|       Maine|      104|US-Canada Border|01-2024|              Trucks|          6556|  45.806|  -70.397|
|     Porthill|       Idaho|     3308|US-Canada Border|04-2024|              Trucks|            98|    49.0| -116.499|
|     San Luis|     Arizona|     2608|US-Mexico Border|04-2024|               Buses|            10|  32.485| -114.782|
| Willow Creek|     Montana|     3325|US-Canada Border|01-2024|         Pedestrians|             2|    49.0| -109.731|
|      Warroad|   Minnesota|     3423|US-Canada Border|01-2024|Personal Vehicle ...|          9266|  48.999|  -95.377|
|     Whitlash|     Montana|     3321|US-Canada 

In [98]:
query = """
    SELECT DISTINCT Date
    FROM BORDER_TABLE
"""

spark.sql(query).show(50, False)

+-------+
|Date   |
+-------+
|08-2006|
|06-2023|
|04-2002|
|09-2000|
|11-2010|
|07-1998|
|09-2023|
|10-2020|
|02-2009|
|11-2019|
|01-1999|
|12-2015|
|05-2013|
|07-2010|
|05-2021|
|03-2007|
|12-2008|
|04-2000|
|08-2022|
|04-2022|
|01-2015|
|01-2018|
|05-2009|
|11-2001|
|07-2000|
|05-2016|
|04-2006|
|11-1998|
|09-2020|
|09-2006|
|06-1999|
|02-2007|
|05-1998|
|04-2013|
|03-2006|
|12-2004|
|03-2019|
|09-2010|
|05-2018|
|06-2007|
|04-2015|
|06-2011|
|10-1999|
|10-2002|
|01-2007|
|11-2021|
|07-2017|
|07-1999|
|05-1996|
|07-2021|
+-------+
only showing top 50 rows



In [99]:
df_2.show()

+-------------+------------+---------+----------------+-------+--------------------+--------------+--------+---------+
|    Port_Name|       State|Port_Code|          Border|   Date|             Measure|Crossing_Count|Latitude|Longitude|
+-------------+------------+---------+----------------+-------+--------------------+--------------+--------+---------+
|      Jackman|       Maine|      104|US-Canada Border|01-2024|              Trucks|          6556|  45.806|  -70.397|
|     Porthill|       Idaho|     3308|US-Canada Border|04-2024|              Trucks|            98|    49.0| -116.499|
|     San Luis|     Arizona|     2608|US-Mexico Border|04-2024|               Buses|            10|  32.485| -114.782|
| Willow Creek|     Montana|     3325|US-Canada Border|01-2024|         Pedestrians|             2|    49.0| -109.731|
|      Warroad|   Minnesota|     3423|US-Canada Border|01-2024|Personal Vehicle ...|          9266|  48.999|  -95.377|
|     Whitlash|     Montana|     3321|US-Canada 

In [100]:
data = df_2.toPandas()
data.to_csv('Border_Crossing_Entry_Data_Final.csv', index=False)